In [ ]:
import os
import torch.nn.functional as F
import torch
from torch.utils.tensorboard import SummaryWriter
from torch import optim
from sklearn import metrics
import numpy as np
import cv2
sys.path.append('../')
from config import *
from file_helper import *

#======= [ SETTING ] =======
MAX_PREDICTION_COUNT = Horizon_MAX_PREDICTION
BATCH_SIZE=6
C = Horizon_C
R = Horizon_R
CONFIDENCE_THRESHOLD = Horizon_CONFIDENCE_THRESHOLD
DO_AUG= Horizon_AUG

#MODEL_FOLDER =r'./output/'
MODEL_FOLDER = create_folder( os.getcwd() , "output" , "checkpoints" )

LONAD_MODEL_NAME =""  
TRAIN_NAME = f"__Test"
writer = SummaryWriter(TRAIN_NAME)

TRAIN_DATASET_NAME  = "../../train_visiable_20_no_cross.json"
TEST_DATASET_NAME   = "../../test_visiable_10_no_cross.json"

#========= [ Log Setting ] ==========
MAX_LOG_GAP = 5
MAX_LOG_IT_COUNT = 5
EVAL_GAP = 5
save_auc = 0.2
log_folder = create_folder( os.path.join(os.getcwd() , "output" , TRAIN_NAME ))

In [ ]:
#======= [ Load Model ] =======
from horizon_model_direct import HorizonNet
def create_model(load_pth = ""):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    net = HorizonNet('resnet50', True , MAX_PREDICTION_COUNT).to(device)   # For server (small memory)

    if (load_pth is not "" or None):              
        model_path = os.path.join(MODEL_FOLDER , load_pth)
        state_dict = torch.load(model_path, map_location='cpu')
        
        # model data
        net.load_state_dict(state_dict['state_dict'])
        LOADED_EPOCH    = state_dict['epoch']
        LOADED_AP       = state_dict['ap']
        save_auc        = LOADED_AP  
        print( LOADED_EPOCH , LOADED_AP)
    return net

net = create_model(LONAD_MODEL_NAME)
optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, net.parameters()),
        lr=1e-4, betas=(0.9, 0.999))